# MAD Dynamique <br>
### STAGE ANCHES <br>
Camille Urban <br>
22/03/2024

In [3]:
# classic shit
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy as sp
from scipy.signal import butter, filtfilt

#librosa
import librosa
import librosa.display

# traitement image
from skimage.io import imread
from skimage.transform import rotate
# from im_rotate import im_rotate
import os

In [4]:
%matplotlib
%matplotlib

Using matplotlib backend: TkAgg
Using matplotlib backend: TkAgg


## Paramètres du Banc

In [66]:
# SENSIBILITE CAPTEURS
S_mic_int =  0.0841  # Sensibilité du capteur de pression interne (mV/Pa)
Offset_mic_int = 51.2   # (mV) 
# G_micro_int = 10 V
S_force = 39.2 # mV/N
# Offset_force = 
# G_force = 10 V
# étalonnage de micro exterieur par source étalon 1kHz à 97dB

# FRQ ECHANTILLONNAGE
sr = 10000 
n_fft = 512 # env. 50ms
# hop_length = n_fft / 4  # = 12.5ms (pas de décalage de la fenêtre)
hop_length = n_fft / 4
p_ref = 2e-5

largeur_bande = 100


In [67]:
dossier = 'mes_2024.04.19/'
nom_fichier = 'anche'

data = pd.read_table(dossier + nom_fichier + '2', sep='\t', header=2).values
tps = data[:,0]
force = data[:,1]/S_force
p_int = (data[:,2])/S_mic_int
# print(type(p_int))
p_ext = data[:,3]
# vib = data[:,4]


nombre_anches = 5


# Traitement données

### Normalisation

In [68]:
# NORMALISATION
 
def normalisation(signal):
    # normalized_signal = signal / np.mean(signal)
    normalized_signal = signal / np.max(signal)
    return normalized_signal


plot_fig = False
plt.close('all')
signal = p_int
if plot_fig :
    plt.figure()
    plt.plot(tps, signal, label='Signal initial')
    plt.plot(tps, normalisation(signal), label='Signal normalisé')
    plt.xlabel('Temps (s)')
    plt.ylabel('Amplitude')
    # plt.title('')
    plt.legend()
    plt.show()


### Fréquence fondamentale 

In [69]:
print_frq = True

def freq_fond(signal, sr) :  
    TF = np.fft.fft(signal)
    freqs = np.fft.fftfreq(len(signal), d=1/sr)         # valeurs des frequences du spectre 
    indice_fonda = np.argmax(np.abs(TF[1:])) + 1        # indice de la frequence fondamentale (+1 pour éviter l'offset en 0)
    freq_fonda = np.abs(freqs[indice_fonda])            # fréquence fondamentale
    if print_frq : 
        print("Fréquence fondamentale :", freq_fonda)
    return freq_fonda


### Filtrer le signale fréquentiellement autour de la fondamentale

In [73]:
# Retourne le signal filtré autour de la fréquence fondamentale
# Prend en entrée le signal et la largeur de la bande passante autour de la fréquence fondamentale
# largeur de la bande frequentielle - freq_bande = 100   
def filtered_sig(signal, sr, largeur_bande):
    freq_fonda = freq_fond(signal, sr)
    
    # Definition de l'ordre du filtre
    ordre_filtre = 4  
    
    # Calcul des fréquences de coupure
    freq_coup_inf = freq_fonda - largeur_bande / 2
    freq_coup_sup = freq_fonda + largeur_bande / 2

    # fréquences de coupure en fréquences normalisées
    frequence_coupure_inf_norm = freq_coup_inf / (sr / 2)
    frequence_coupure_sup_norm = freq_coup_sup / (sr / 2)

    # Création du filtre passe-bande
    b, a = butter(ordre_filtre, [frequence_coupure_inf_norm, frequence_coupure_sup_norm], btype='band')

    # Filtrage du signal
    signal_filtre = filtfilt(b, a, signal)
    return signal_filtre


plot_fig = False
plt.close('all')
signal = p_int
if plot_fig :
    plt.figure()
    plt.plot(tps, normalisation(signal), label='Signal normalisé')
    plt.plot(tps, filtered_sig(normalisation(signal), sr=sr, largeur_bande=largeur_bande), label='Signal normalisé filtré')
    plt.xlabel('Temps (s)')
    plt.ylabel('Amplitude normalisée')
    plt.title('Comparaison entre le signal initial et le signal filtré')
    plt.legend()
    plt.show()

### Enveloppe Spectrale

In [80]:
# Fonction pour calculer l'enveloppe spectrale par convolution avec fenêtre de Hann
def spectral_envelope(signal, window_size): 
    # Calculer le nombre de segments
    num_segments = len(signal) // window_size
    
    # Initialiser l'enveloppe spectrale
    spectral_env = np.zeros(len(signal))
    
    # fenêtrage de Hann sur chaque segment et TF
    for i in range(num_segments):
        start_idx = i * window_size
        end_idx = (i + 1) * window_size
        segment = signal[start_idx:end_idx]
        
        # fenêtrage
        segment *= sp.signal.windows.hann(window_size)
        
        # TF et amplitude
        spectrum = np.fft.fft(segment)
        amplitude = np.abs(spectrum)
        
        spectral_env[start_idx:end_idx] = amplitude
    
    # Lissage par convolution
    spectral_env = np.convolve(spectral_env, np.ones(window_size) / window_size, mode='same')
    
    return spectral_env

plot_fig = False
plt.close('all')
signal = p_int
if plot_fig :
    plt.figure()
    plt.plot(tps, filtered_sig(normalisation(signal), sr=sr, largeur_bande=largeur_bande), label='Signal normalisé filtré')
    plt.plot(tps, spectral_envelope(filtered_sig(normalisation(signal), sr=sr, largeur_bande=largeur_bande), window_size=n_fft), label='Enveloppe spectrale')
    plt.xlabel('Temps (s)')
    plt.ylabel('Amplitude normalisée')
    plt.title('Enveloppe spectrale du signal filtré autour de la fréquence fondamentale')
    plt.legend()
    plt.show()

### Seuillage

In [93]:
# Détermination du seuil de détéction paour chaque l'enveloppe spectrale du signal normalisé et filtré autour de f_0
# On prend le seuillage sur 100ms 
print_seuil = True

def seuillage(signal, sr, largeur_bande, window_size):
    signal = normalisation(signal)
    signal = filtered_sig(signal, sr, largeur_bande)
    signal = spectral_envelope(signal, window_size)
    
    longueur_selection = int(0.1*sr) 
    selection = signal[:longueur_selection]
    moyenne = np.mean(selection)
    e_type = np.std(selection)
    seuil = moyenne + 2* e_type 
    if print_seuil:
        print(seuil)
    return seuil

plot_fig = True
plt.close('all')
signal = p_int
if plot_fig :
    plt.figure()
    plt.plot(tps, filtered_sig(normalisation(signal), sr=sr, largeur_bande=largeur_bande), label='Signal normalisé filtré')
    plt.plot(tps, spectral_envelope(filtered_sig(normalisation(signal), sr=sr, largeur_bande=largeur_bande), window_size=n_fft), label='Enveloppe spectrale')
    plt.hlines(seuillage(signal, sr=sr, largeur_bande=largeur_bande, window_size=n_fft), tps[0], tps[-1], color='yellow', label='Signal normalisé filtré')
    plt.xlabel('Temps')
    plt.ylabel('Amplitude')
    plt.title('Comparaison entre le signal initial et le signal filtré')
    plt.legend()
    plt.show()

Fréquence fondamentale : 209.6620570753409
Fréquence fondamentale : 209.6620570753409
Fréquence fondamentale : 209.6620570753409
0.0031222343105706866


### Troncatures

In [108]:
def troncature(signal, sr, largeur_bande, window_size) : 
    # Signal filtré
    signal_filtre = filtered_sig(signal, sr, largeur_bande)
    # Enveloppe spectrale
    env_spec = spectral_envelope(signal_filtre, window_size)
    # Normalisation de l'enveloppe
    env_spec = spectral_envelope(signal_filtre, window_size) / np.max(spectral_envelope(signal_filtre, window_size))

    # Seuil 
    seuil = seuillage(signal, sr, largeur_bande, window_size)
    
    idx_tr =[]
    seuil_depasse = False
    for i, valeur in enumerate(env_spec):
        if not seuil_depasse and valeur > seuil:
            seuil_depasse = True
            idx_tr.append(i)  # Ajouter le moment où le seuil est dépassé
        elif seuil_depasse and valeur <= seuil:
            seuil_depasse = False
            idx_tr.append(i)   
           
    # # Tronquer le signal autour des indices de détection de seuil
    # idx_tr1_deb = idx_tr[0]
    # idx_tr1_fin = idx_tr[1]
    # idx_tr2_deb = idx_tr[2]
    # idx_tr2_fin = idx_tr[-1]
    
    # Calculer la somme cumulée des énergies normalisées
    e_cum = np.cumsum(env_spec)

    # passage de seuil
    idx_sec = []
    
    #Faire la division avec calcul energie pour chaque sous section
    print(idx_sec)
    e_cum = normalisation(e_cum)
    idx = np.array(idx_tr[0], idx_sec[0], idx_sec[1], idx_tr[1], idx_tr[2], idx_sec[2], idx_sec[3], idx_tr[-1])
    
    return idx


troncature(signal=p_int, sr=sr, largeur_bande=100, window_size=n_fft)

Fréquence fondamentale : 209.6620570753409
Fréquence fondamentale : 209.6620570753409
0.0031222343105706866
[]


IndexError: list index out of range

# Obsolet

### Extraction de données et plots temporels

In [1]:
# LECTURE ET PLOT DES DONNEES NORMALISEES

def read_datas(nbre_anches) :
    temps = []
    forces = []
    p_ints = []
    p_exts = []
    vibs = []
    for i in range (nbre_anches):
        data = pd.read_table( dossier + nom_fichier + f'{i+1}', sep='\t', header=2).values
        tps = data[:,0]
        # tps = normalisation(tps)
        temps.append(tps)
        force = data[:,1]/S_force
        force = normalisation(force)
        forces.append(force)
        p_int = (data[:,2])/S_mic_int
        p_int = normalisation(p_int)
        p_ints.append(p_int)
        p_ext = data[:,3]
        p_ext = normalisation(p_ext)
        p_exts.append(p_ext)
        # vib = data[:,4]
        # vib = normalisation(vib)
        # vibs.append(vib)
    return temps, forces, p_ints, p_exts
    # return temps, forces, p_ints, p_exts, vib

temps, forces, p_ints, p_exts = read_datas(5)

def plot_signaux_init(nbre_anches):
    plt.close('all')
    temps, forces, p_ints, p_exts = read_data(nbre_anches)
    for i in range(nbre_anches):
        # configuration d'afficage
        fig, axs = plt.subplots(2, 2, gridspec_kw={'width_ratios': [1, 1], 'height_ratios': [1, 1]})
        fig.subplots_adjust(left=0.1, right=0.9, bottom=0.1, top=0.9, wspace=0.4, hspace=0.4)
        fig.suptitle(f'Signaux normalisés \nAnche n° {i}', fontsize=16)
        # données à afficher
        axs[0,0].plot(temps[i], forces[i], color='red')
        axs[0,0].set_title('Force (N)', fontsize=14)
        # axs[0,1].plot(temps[i], vib[i], color='green')
        axs[0,1].set_title('Déplacement (mm)', fontsize=14)
        axs[1,1].plot(temps[i], p_exts[i], color='purple')
        axs[1,1].set_title('Pression externe (Pa)', fontsize=14)
        axs[1,0].plot(temps[i], p_ints[i], )
        axs[1,0].set_title('Pression interne (Pa)', fontsize=14)
        # définition axes
        for ax in axs.flat:
            ax.set_xlabel('Temps (s)', fontsize=12)
            ax.set_ylabel('Amplitude (Pa)', fontsize=12)
            ax.grid(True)
        
        # plt.tight_layout()
        plt.show()

plot_signaux_init(5)

NameError: name 'pd' is not defined

### Spectrogramme des signaux en pression (microphone interne et externe)

In [25]:
# Retourne les sepctrogramme sur la pression interne et externe en dB et les plot
def plot_spectro(nbre_anches, n_fft):
    temps, forces, p_ints, p_exts = read_data(nbre_anches) 
    for i in range (nbre_anches):
        # Spectrogramme
        spectrogram_int = np.abs(librosa.stft(y=p_ints[i], n_fft=n_fft))
        spectrogram_ext = np.abs(librosa.stft(y=p_exts[i], n_fft=n_fft))
        # Mise en dB
        spectrogram_int_db = librosa.power_to_db(spectrogram_int**2, ref=p_ref**2)
        spectrogram_ext_db = librosa.power_to_db(spectrogram_ext**2, ref=p_ref**2)
        
        # Affichages des spectrogrammes
        fig, (pres_int, pres_ext) = plt.subplots(1, 2, figsize=(10, 4))
        fig.subplots_adjust(left=0.1, right=0.9, bottom=0.1, top=0.9, wspace=0.4, hspace=0.4)
        fig.suptitle(f'Sepctrogrammes \nAnche n° {i}', fontsize=16)
        
        img_int = librosa.display.specshow(spectrogram_int_db, sr=sr, hop_length=hop_length, x_axis='time', y_axis='mel', ax=pres_int)
        cbar = fig.colorbar(img_int, ax=pres_int, format='%+2.0f dB')
        cbar.set_label('Amplitude (dB)')
        pres_int.set_title('Pression interne au bec')

        img_ext = librosa.display.specshow(spectrogram_ext_db, sr=sr, hop_length=hop_length, x_axis='time', y_axis='mel', ax=pres_ext)
        cbar = fig.colorbar(img_ext, ax=pres_ext, format='%+2.0f dB')
        cbar.set_label('Amplitude (dB)')
        pres_ext.set_title('Pression externe au bec')

        plt.tight_layout()
        plt.show()
    return spectrogram_int_db, spectrogram_ext_db

plt.close('all')       
spectrogram_int_db, spectrogram_ext_db = plot_spectro(5, n_fft)


In [27]:
# Retourne le signal filtré autour de la fréquence fondamentale
# Prend en entrée le signal et la largeur de la bande passante autour de la fréquence fondamentale
def filtered_sig(signal, largeur_bande) : # prendre sr
    freq_fonda = freq_fond(signal)
    # print(freq_fonda)
    
    # Definition de l'ordre du filtre
    ordre_filtre = 4  
    
    # Calcul des fréquences de coupure
    freq_coup_inf = freq_fonda - largeur_bande / 2
    freq_coup_sup = freq_fonda + largeur_bande / 2

    # fréquences de coupure en fréquences normalisées
    frequence_coupure_inf_norm = freq_coup_inf / (sr / 2)
    frequence_coupure_sup_norm = freq_coup_sup / (sr / 2)

    # Création du filtre passe-bande
    b, a = butter(ordre_filtre, [frequence_coupure_inf_norm, frequence_coupure_sup_norm], btype='band')

    # Filtrage du signal
    signal_filtre = filtfilt(b, a, signal)
    return signal_filtre


### Enveloppe spectrale

In [1]:
# Fonction pour calculer l'enveloppe spectrale par convolution avec fenêtre de Hann
def spectral_envelope(signal, window_size): # prendre sr
    # Calculer le nombre de segments
    num_segments = len(signal) // window_size
    
    # Initialiser l'enveloppe spectrale
    spectral_env = np.zeros_like(signal) 
    
    # fenêtrage de Hann sur chaque segment et TF
    for i in range(num_segments):
        start_idx = i * window_size
        end_idx = (i + 1) * window_size
        segment = signal[start_idx:end_idx]
        
        # fenêtrage
        segment *= sp.signal.windows.hann(window_size)
        
        # TF et amplitude
        spectrum = np.fft.fft(segment)
        amplitude = np.abs(spectrum)
        
        spectral_env[start_idx:end_idx] = amplitude
    
    # Lissage par convolution
    spectral_env = np.convolve(spectral_env, np.ones(window_size) / window_size, mode='same')
    
    return spectral_env


#### Seuillage

In [2]:
# Détermination du seuil 
# On prend le seuillage sur 100ms 
def seuillage(signal):
    longueur_selection = int(0.1*sr) 
    selection = signal[:longueur_selection]
    maximum = np.max(selection)
    moyenne = np.mean(selection)
    e_type = np.std(selection)
    seuil = moyenne + 2* e_type 
    # seuil = maximum + moyenne
    return seuil

In [3]:
plt.close('all')

freq_bande = 100    # largeur de la bande frequentielle

# Taille de la fenêtre de Hann
window_size = n_fft
# F_c = 20  # Fréquence de coupure (Hz)
# window_size = int(sr / F_c)   # cf. 2017, Petiot

for n in range(nombre_anches) : 
    signal = p_ints[n]
    tps = temps[n]

    seuil = seuillage(signal)
    print(seuil)
    
    # SIGNAL FILTRE
    signal_filtre = filtered_sig(signal, freq_bande)
    # ENVELOPPE
    spectral_env = spectral_envelope(signal_filtre, window_size)
    # Normalisation de l'enveloppe
    spectral_env_normalized = spectral_env / np.max(spectral_env)

    # Afficher le signal et son enveloppe spectrale
    plt.figure(figsize=(10, 6))
    # plt.plot(tps, signal, label='Signal normalisé')           # signal brut
    # plt.plot(tps, np.abs(signal), label='Signal normalisé')   # val absolue du signal
    plt.plot(tps, signal_filtre, label='Signal normalisé')      # signal filtre autour de la ère freq fonda
    plt.plot(tps, spectral_env_normalized, label='Enveloppe spectrale')
    plt.hlines(seuil, tps[0], tps[-1], color='yellow' ,label=f'seuil {round(seuil,3)}')
    plt.xlabel('Temps (s)')
    plt.ylabel('Amplitude normalisée')
    plt.title('Enveloppe spectrale par convolution avec fenêtre de Hann')
    plt.legend()
    plt.grid(True)
    plt.show()

NameError: name 'plt' is not defined

In [41]:
plt.close('all')
# tracer le signal initial et le signal filté
plt.figure()
plt.plot(tps, signal, label='Signal initial')
plt.plot(tps, signal_filtre, label='Signal filtré')
plt.xlabel('Temps')
plt.ylabel('Amplitude')
plt.title('Comparaison entre le signal initial et le signal filtré')
plt.legend()
plt.show()

### Troncature signaux

In [101]:
def troncature(nombre_anches, window_size) : 
    temps, forces, p_ints, p_exts = read_data(nombre_anches)
    sig_tronc = np.empty((2, 2), dtype=object)
    for n in range(nombre_anches) :
        segments = []
        signal = p_ints[n]
        tps = temps[n]
        
        # SIGNAL FILTRE
        signal_filtre = filtered_sig(signal, freq_bande)
        # ENVELOPPE
        # spectral_env = spectral_envelope(signal, window_size)
        spectral_env = spectral_envelope(signal_filtre, window_size)

        # Normalisation de l'enveloppe
        spectral_env_normalized = spectral_env / np.max(spectral_env)
        
        # Seuil 
        seuil = seuillage(signal)
        seuil_depasse = False
        for i, valeur in enumerate(spectral_env_normalized):
            if not seuil_depasse and valeur > seuil:
                seuil_depasse = True
                segments.append(i)  # Ajouter le moment où le seuil est dépassé
            elif seuil_depasse and valeur <= seuil:
                seuil_depasse = False
                segments.append(i)   

        print(segments)
        # Tronquer le signal autour des indices de détection de seuil
        sig_tronc[0,0] = tps[segments[0] :segments[1]]
        sig_tronc[0,1] = signal[segments[0] :segments[1]]
        # Tronquer le signal autour des indices de détection de seuil
        sig_tronc[1,0] = tps[segments[2] : segments[-1]]
        sig_tronc[1,1] = signal[segments[2] :segments[-1]]
        # Calculer la somme cumulée des énergies normalisées
        e_cum = np.cumsum(spectral_env_normalized)

        # passage de seuil
        passage_seuil = []
        
        # Parcourir la somme cumulée des énergies
        for i in range(1, len(e_cum)):
            # Si la somme cumulée dépasse le seuil absolu
            if e_cum[i] >= seuil and e_cum[i - 1] < seuil:
                passage_seuil.append(i)  # Ajouter l'indice où la somme d'énergie cumulée passe au-dessus du seuil
            # Si la somme cumulée passe en dessous du seuil absolu
            elif e_cum[i] < seuil and e_cum[i - 1] >= seuil:
                passage_seuil.append(i)

        e_cum = normalisation(e_cum)

    return seuil, segments, sig_tronc, e_cum


seuil, segments, sig_tronc, e_cum = troncature(5, n_fft)
print(sig_tronc)

plt.figure(figsize=(10, 6))
plt.plot(tps, signal_filtre, label='Signal filtré normalisé')
plt.plot(tps, spectral_env_normalized, label='Enveloppe spectrale')
plt.plot(tps, e_cum, label='somme cumulée des énergie')
plt.grid(True)
plt.xlabel('Temps (s)')
plt.ylabel('Energie')
plt.title('Energie cumulative')
plt.legend()

plt.figure(figsize=(10, 6))
plt.plot(sig_tronc[0,0], sig_tronc[0,1], label='Signal normalisé')
plt.grid(True)
plt.xlabel('Temps (s)')
plt.ylabel('Amplitude normalisée')
plt.title('Signal tronqué - pression croissante')
plt.legend()

plt.figure(figsize=(10, 6))
plt.plot(sig_tronc[1,0], sig_tronc[1,1], label='Signal normalisé')
plt.grid(True)
plt.xlabel('Temps (s)')
plt.ylabel('Amplitude normalisée')
plt.title('Signal tronqué - pression croissante')
plt.legend()


NameError: name 'read_data' is not defined

In [79]:
plt.close('all')

plot_fig = False
def signal_statio(temps, signal) : 
    E_t = sp.integrate.cumtrapz(y=signal**2, x=temps, dx = 1/sr)    #Cumulatively integrate
    
    indices_statio = [np.argmax(E_t >= 0.05 * np.max(E_t)), np.argmax(E_t > 0.95 * np.max(E_t))-1 ]
    # tps_statio = temps[indices_statio]
    # E_t_statio = E_t[indices_statio]
    print(indices_statio)
    # print(tps_statio)
    # print(E_t_statio)
    return indices_statio

# signals_troncs = [sig_tronc[0,1], sig_tronc[1,1]]
# temps_troncs = [sig_tronc[0,0], sig_tronc[1,0]]
labels = ['Début du signal staionnaire \n$0.05*E(t)$', 'Fin du signal staionnaire \n$0.95*E(t)$']
color = ['orange', 'green']

for i in range(2):
    temps_part = sig_tronc[i,0]
    sig_part = sig_tronc[i,1]
    indices_statio = signal_statio(temps_part, sig_part)
    
    
    if plot_fig : 
        plt.figure(figsize=(10, 6))
        plt.plot(temps_part, sig_part, label='Energie cumulée') 
    for j in range (2): 
        x_j = temps_part[indices_statio[j]]
        y_j = sig_part[indices_statio[j]]
        if plot_fig : 
            plt.vlines(x_j, -1, 1, colors=color[j], label=labels[j])
            # plt.hlines(y_j, temps_part[0], temps_part[-1], colors=color[j])
            # plt.scatter(x_j, y_j, color=color[i])
            plt.text(x_j, 0.85, f'({round(x_j,2)})', fontsize=10)
            plt.grid(True)
            plt.xlabel('Temps (s)')
            plt.ylabel('Amplitude normalisée')
            plt.title('Energie en fonction du temps $E(t) = \int_{t_{deb}}^{t} p^{2}(\\tau) d\\tau$')
            plt.legend()
 

[3278, 29489]
[1817, 14537]


C:\Users\Admin\AppData\Local\Temp\ipykernel_20336\175972585.py:5: DeprecationWarning: 'scipy.integrate.cumtrapz' is deprecated in favour of 'scipy.integrate.cumulative_trapezoid' and will be removed in SciPy 1.14.0
  E_t = sp.integrate.cumtrapz(y=signal**2, x=temps, dx = 1/sr)    #Cumulatively integrate


In [64]:
plt.close('all')

signals_troncs = [sig_tronc[0,1], sig_tronc[1,1]]
temps_troncs = [sig_tronc[0,0], sig_tronc[1,0]]
pression = ['croissante', 'decroissante']
   
for i in range(2):
    xi = temps_troncs[i]
    yi = signals_troncs[i]
    E_t, indices_statio, tps_statio, E_t_statio = signal_statio(xi, yi)
    for j in range (2): 
        x = tps_statio[j]
        y = E_t_statio[j]
        print(type(y))
        
    plt.figure(figsize=(10, 6))
    plt.title(f'Section transitoire pression seuil - pression {pression[i]}')
    plt.plot(xi[0:indices_statio[0]], yi[0:indices_statio[0]], label='Section 1', color='red')
    plt.figure(figsize=(10, 6))
    plt.title(f'Section Stationnaire - pression {pression[i]}')
    plt.plot(xi[indices_statio[0]:indices_statio[1]], yi[indices_statio[0]:indices_statio[1]], label='Section 2', color='green')
    plt.figure(figsize=(10, 6))
    plt.title(f'Section transitoire pression de plaquage - pression {pression[i]}')
    plt.plot(xi[indices_statio[1]:-1], yi[indices_statio[1]:-1], label='Section 3', color='orange')

    plt.grid(True)
    plt.xlabel('Temps (s)')
    plt.ylabel('Amplitude normalisée')
    plt.legend()


C:\Users\Admin\AppData\Local\Temp\ipykernel_20336\3282147327.py:5: DeprecationWarning: 'scipy.integrate.cumtrapz' is deprecated in favour of 'scipy.integrate.cumulative_trapezoid' and will be removed in SciPy 1.14.0
  E_t = sp.integrate.cumtrapz(y=signal**2, x=temps, dx = 1/sr)    #Cumulatively integrate


<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>


In [6]:
# descritpeurs section stationnaire 
plt.close('all')

signals_troncs = [sig_tronc[0,1], sig_tronc[1,1]]
temps_troncs = [sig_tronc[0,0], sig_tronc[1,0]]
pression = ['croissante', 'decroissante']
   
for i in range(2):
    t_i = temps_troncs[i]
    sig_i = signals_troncs[i]
    E_t, indices_statio, tps_statio, E_t_statio = signal_statio(xi, yi)
    for j in range (2): 
        x = tps_statio[j]
        y = E_t_statio[j]
        # print(type(y))
    
    t_sta = xi[indices_statio[0]:indices_statio[1]]
    print(len(t_sta))
    sig_sta = yi[indices_statio[0]:indices_statio[1]]
    print(len(sig_sta))
    plt.figure(figsize=(10, 6))
    plt.title(f'Section Stationnaire - pression {pression[i]}')
    plt.plot(t_sta, sig_sta, label='Section 2', color='green')

    plt.grid(True)
    plt.xlabel('Temps (s)')
    plt.ylabel('Amplitude normalisée')
    plt.legend()
    
    
    CGS_int = librosa.feature.spectral_centroid(y=sig_sta, sr=sr, n_fft=n_fft).squeeze()
    n=min(len(t_sta), len(CGS_int))
    print(n)
    print(CGS_int.shape)
    plt.figure()
    plt.title(f'Section Stationnaire - pression {pression[i]}')
    plt.plot(t_sta[:n], CGS_int[:n], label='CGS')
    plt.xlabel('Time')
    plt.ylabel('CGS (Hz)')
    plt.legend()

NameError: name 'sig_tronc' is not defined

#### En fait non

In [18]:
# ETUDES SUR PARTIE STATIONNAIRE
plt.close('all')

sign_statio = []
temps_statio = []
CGSs = []

for i in range(2):
    plt.figure()
    x = temps_troncs[i]
    y = signals_troncs[i]
    _, indices_statio, _, _ = signal_statio(x, y)
    x = x[indices_statio[0]:indices_statio[1]]
    # temps_statio.append(x)
    y = y[indices_statio[0]:indices_statio[1]]
    # sign_statio.append(y)
    CGS = librosa.feature.spectral_centroid(y=y, sr=sr, n_fft=n_fft).squeeze()
    t_max = len(y)//sr
    N = len(y)//n_fft
    print(N)
    t = np.linspace(0, t_max, N+1)
    print(len(t))
    print(len(CGS))

    plt.plot(t, CGS, label='ext')
    
plt.xlabel('Time')
plt.ylabel('CGS (Hz)')
plt.legend()

51
52
52
24
25
25


C:\Users\Admin\AppData\Local\Temp\ipykernel_20336\3285776097.py:5: DeprecationWarning: 'scipy.integrate.cumtrapz' is deprecated in favour of 'scipy.integrate.cumulative_trapezoid' and will be removed in SciPy 1.14.0
  E_t = sp.integrate.cumtrapz(y=signal**2, x=temps, dx = 1/sr)


In [19]:
plt.close('all')

# y = p_ext
# S, phase = librosa.magphase(librosa.stft(y)) # S=p_int

rms = librosa.feature.rms(S=spectrogram_int, frame_length = n_fft).squeeze() # A mettre en dB 
print(rms.shape)
rms_diff = np.diff(rms, 1)
max_diff = np.abs(np.max(rms_diff))
ind = np.where(rms_diff>0.5*max_diff)[0]
print(ind*hop_length/sr)
seuil_0_diff = 0.00005
ind2 = np.where(np.abs(rms_diff)<seuil_0_diff)[0]
print(ind2*hop_length/sr)

plt.figure()

times = librosa.times_like(rms)
plt.plot(times, rms, label='RMS Energy')
times = librosa.times_like(rms_diff)
plt.plot(times, rms_diff, label='RMS Energy')
plt.legend()

# librosa.display.specshow(spectrogram_int_db, y_axis='linear', x_axis='time', ax=ax[1])

# ax[2].plot(tps, A[:,3], label='micro bec')

NameError: name 'spectrogram_int' is not defined

In [ ]:
plt.close('all')

# y = p_ext
# S, phase = librosa.magphase(librosa.stft(y)) # S=p_int

rms = librosa.feature.rms(S=spectrogram_int, frame_length = n_fft).squeeze() # A mettre en dB 
print(rms.shape)
rms_diff = np.diff(rms, 1)
max_diff = np.abs(np.max(rms_diff))
ind = np.where(rms_diff>0.5*max_diff)[0]
print(ind*hop_length/sr)
seuil_0_diff = 0.00005
ind2 = np.where(np.abs(rms_diff)<seuil_0_diff)[0]
print(ind2*hop_length/sr)

fig, ax = plt.subplots(nrows=2, sharex=True)

times = librosa.times_like(rms)
ax[0].plot(times, rms, label='RMS Energy')
times = librosa.times_like(rms_diff)
ax[0].plot(times, rms_diff, label='RMS Energy')
ax[0].set(xticks=[])
ax[0].legend()
ax[0].label_outer()

librosa.display.specshow(spectrogram_int_db, y_axis='linear', x_axis='time', ax=ax[1])
ax[1].set(title='log Power spectrogram')

# ax[2].plot(tps, A[:,3], label='micro bec')

(648,)
[2.0224 4.608  4.6208 4.6336 4.6464]
[0.256  0.4736 0.5632 0.6144 0.7168 0.7936 0.8576 0.9472 1.024  1.088
 1.1264 1.1904 1.2288 1.2416 1.3056 2.944  3.8528 4.224  4.2752 4.352
 4.5184 6.3872 6.5792 6.6816 6.912  6.9504 7.04   7.0528 7.1168 7.2064
 7.3216 7.3344 7.4496 7.4752 7.5136 7.6032 7.7184 7.7312 7.808  7.8336
 7.8592 7.872  7.8976 7.9232 7.9488 7.9872 8.     8.0256 8.0512 8.064
 8.0896 8.1536 8.1664 8.1792 8.2176 8.256  8.2688]


[Text(0.5, 1.0, 'log Power spectrogram')]

In [81]:
plt.close('all')
plt.figure()

# on peut aussi demander à librosa de traduire notre temps (tps) en frame directement
# times = librosa.times_like(CGS_int)
print(len(tps))
# t = librosa.time_to_frames(tps, sr=sr, hop_length=n_fft//4)
# print(t.shape)
t_max = len(p_int)//sr
N = len(p_int)//n_fft
print(N)
t = np.linspace(0, t_max, N)



CGS_int = librosa.feature.spectral_centroid(y=p_int, sr=sr, n_fft=n_fft).squeeze()
# CGS_ext = librosa.feature.spectral_centroid(y=p_ext, sr=sr, n_fft=n_fft).squeeze()
n=min(len(t), len(CGS_int))
print(n)

print(CGS_int.shape)
plt.plot(t[:n], CGS_int[:n], label='int')
# plt.plot(t[:n], CGS_ext[:n], label='ext')
plt.xlabel('Time')
plt.ylabel('CGS (Hz)')
plt.legend()
# reg = LinearRegression()
# reg.fit(force,CGS)
# reg.score(X,CGS)

90111


NameError: name 'p_int' is not defined

In [ ]:

plt.close('all')
fig, axs = plt.subplots(3, 2, figsize=(12, 7))

# FFT
fft_int = np.fft.fft(p_int)
fft_freq_int = np.fft.fftfreq(p_int.shape[-1])
axs[0, 0].plot(fft_freq_int, fft_int)
axs[0, 0].set_xlabel('Frequence (Hz)')
axs[0, 0].set_ylabel('Amplitude (mV)')
axs[0, 0].set_title('Intérieur Bec \n FFT')
axs[0, 0].set_xlim([0, 0.4])
axs[0, 0].set_ylim([0, None])

fft_ext = np.fft.fft(p_ext)
fft_freq_ext = np.fft.fftfreq(p_ext.shape[-1])
axs[0, 1].plot(fft_freq_ext, fft_ext)
axs[0, 1].set_xlabel('Frequence (Hz)')
axs[0, 1].set_ylabel('Amplitude (mV)')
axs[0, 1].set_title('Extérieur Bec \n FFT')
axs[0, 1].set_xlim([0, 0.4])
axs[0, 1].set_ylim([0, None])

# SIGNAUX TEMPORELS
axs[1, 0].plot(tps, p_int, label='micro ext')
S, phase = librosa.magphase(librosa.stft(p_int))
rms = librosa.feature.rms(S=S)
times = librosa.times_like(rms)
# axs[1, 0].plot(times, rms[0], label='RMS Energy')
axs[1, 0].set_title('Signal temporel')
axs[1, 0].set_xlabel('Temps (s)')
axs[1, 0].set_ylabel('Amplitude (mV)')

axs[1, 1].plot(tps, p_ext, label='micro int')
axs[1, 1].set_title('Signal temporel')
axs[1, 1].set_xlabel('Temps (s)')
axs[1, 1].set_ylabel('Amplitude (mV)')


# SPRECTRO
spectrogram_int = librosa.feature.melspectrogram(y=p_int)
spectrogram_ext = librosa.feature.melspectrogram(y=p_ext)

# Convertir l'échelle en décibels (dB)
spectrogram_int_db = librosa.power_to_db(spectrogram_int, ref=np.max)
spectrogram_ext_db = librosa.power_to_db(spectrogram_ext, ref=np.max)

img_int = librosa.display.specshow(spectrogram_int_db, ax=axs[2, 0])
# p_int.set_colorbar(format='%+2.0f dB')
# cbar = fig.colorbar(img_int, ax=axs[2,0], format='%+2.0f dB')
# cbar.set_label('Amplitude (dB)')
axs[2, 0].set_title('Pression interne au bec')
axs[2, 0].set_xlabel('Temps (s)')
axs[2, 0].set_ylabel('Frequence (Hz)')

img_ext = librosa.display.specshow(spectrogram_ext_db, x_axis='time', y_axis='mel', ax=axs[2, 1])
# cbar = fig.colorbar(img_ext, ax=axs[2, 1], format='%+2.0f dB')
cbar.set_label('Amplitude (dB)')
axs[2, 1].set_title('Pression externe au bec')
axs[2, 1].set_xlabel('Temps (s)')
axs[2, 1].set_ylabel('Frequence (Hz)')

# CGS_int = librosa.feature.spectral_centroid(y=p_int)
# CGS_ext = librosa.feature.spectral_centroid(y=p_ext)
# axs[3,0].plot(CGS_int.squeeze())
# axs[3,1].plot(CGS_ext.squeeze())

# fig.suptitle('Spectrogrammes', fontsize=20)
plt.tight_layout()
plt.show()

c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\matplotlib\cbook.py:1699: ComplexWarning: Casting complex values to real discards the imaginary part
  return math.isfinite(val)
c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\matplotlib\cbook.py:1345: ComplexWarning: Casting complex values to real discards the imaginary part
  return np.asarray(x, float)


In [ ]:

# Charger le fichier audio
signal = p_ext
# sr = None

# Extraire les descripteurs audio
energie = librosa.feature.rms(y=signal)
mfcc = librosa.feature.mfcc(y=signal, n_mfcc=13)
chroma = librosa.feature.chroma_stft(y=signal)
zero_crossing_rate = librosa.feature.zero_crossing_rate(y=signal)
spectral_centroid = librosa.feature.spectral_centroid(y=signal)
spectral_contrast = librosa.feature.spectral_contrast(y=signal)
spectral_rolloff = librosa.feature.spectral_rolloff(y=signal)
mfcc_delta = librosa.feature.delta(mfcc)
mfcc_delta_delta = librosa.feature.delta(mfcc, order=2)
centres_bandes = librosa.feature.spectral_centroid(y=signal, n_fft=2048, hop_length=512)

# Afficher les dimensions de chaque descripteur
# print("Énergie du signal:", energie.shape)
print("MFCC:", mfcc.shape)
# print("Chroma:", chroma.shape)
print("Zero-Crossing Rate:", zero_crossing_rate.shape)
print("Spectral Centroid:", spectral_centroid.shape)
print("Spectral Contrast:", spectral_contrast.shape)
print("Spectral Rolloff:", spectral_rolloff.shape)
print("MFCC Delta:", mfcc_delta.shape)
print("MFCC Delta-Delta:", mfcc_delta_delta.shape)
print("Centres de bande de fréquences:", centres_bandes.shape)



Énergie du signal: (1, 194)
MFCC: (13, 194)
Chroma: (12, 194)
Zero-Crossing Rate: (1, 194)
Spectral Centroid: (1, 194)
Spectral Contrast: (7, 194)
Spectral Rolloff: (1, 194)
MFCC Delta: (13, 194)
MFCC Delta-Delta: (13, 194)
Centres de bande de fréquences: (1, 194)
